In [ ]:
# libraries
import pandas as pd
import os
import sys

# modules
sys.path.append(os.path.abspath("modules"))
import modules.dictionary as dict
from modules.classify import classify_keywords

# set your path here
os.chdir("input_data")
print(f'current directory: {os.getcwd()}')

In [ ]:
# target word(s) : class(es) assigned

override_map_2020 = {

    # ----内科・外科----
    ("訪問", "訪問診療", "訪問診療科", "内", "小内", "往診専門", "健康診", "男内", "女内", "内小", "内（薬）", "(新)内", "新生内", "内（兼", "漢方外来", "女性内"): ["一般内科"],
    ("成人病検査のみ", "健康指導", "内（採血）"): ["一般内科"],
    ("漢", "漢方", "漢方専門", "漢方科", "漢方（思）", "漢方（不）", "漢方医学科", "漢方診療科", "漢小", "漢方内", "漢方）"): ["一般内科"], 
    ("総合診療", "総合診療科"): ['一般内科', '一般外科'],
    ("基本診療", "基本診療科"): ['一般内科', '一般外科'],
    ("一般外"): ['一般外科'],

     # ----消化器内科・外科----
    ("消", "消科", "消化", "消化器", "消火器科", "消化器科", "消化器科（胃）", "「消化器", "（消化器）", "消（胃）"): ["消化器内科", "消化器外科"],
    ("内鏡内", "鏡内", "内（鏡）"): ["消化器内科", "消化器外科"],
    ("胃", "胃腸", "胃腸科", "胃腸下科", "胃腸器科", "消・胃腸科", "消胃", "胃腸科（消化器科）", "消化器科∧胃腸科∨"): ["消化器内科", "消化器外科"],
    ("呼胃"): ["呼吸器内科", '呼吸器外科', "消化器内科", "消化器外科"], 
    ("食道科", "食道", "食"): ["消化器内科", "消化器外科"],
    ("消化器・胃腸科", "消化器胃腸科", "消化器科（胃腸科）"): ["消化器内科", '消化器外科'],

     # ----呼吸器内科・外科----
    ("呼", "呼吸", "呼吸器", "呼吸器科", "呼吸科"): ["呼吸器内科", '呼吸器外科'],

    # ----気管食道内科・外科----
    ("気管食道科", "気食", "気管食道", "気管科", "気管支科", "気道食道科", "気管", "喘息外来"): ["気管食道内科", "気管食道外科"],

     # ----循環器内科・外科----
    ("循環器科", "循環器", "循環科", "循", "循環", "心臓", "心血", "心臓血", "心臓血管", "血管", "循小", "（循）小", "循環器）"): ["循環器内科", "循環器外科"],
    ("循呼", "呼循"): ["呼吸器内科", '呼吸器外科', "循環器内科", "循環器外科"],
    ("胃循", "消循"): ["消化器内科", "消化器外科", "循環器内科", "循環器外科"],
    "禁煙外来": ["循環器内科"],

    # ----心臓血管内科・外科----
    ("心血内", "血内"): ["心臓・血管内科"],

    # ----大腸・肛門----
    ("肛内", "大肛内"): ["大腸・肛門内科"],
    ("肛外", "大肛外", "こう外", "大肛", "外肛外", "鏡内肛外"): ["大腸・肛門外科"],
    ("肛","こう門科", "肛門科", "こう門", "大腸肛門科", "外肛内", ""): ["大腸・肛門内科", "大腸・肛門外科"],

    # ----精神・心療----
    "精": ["精神科"],
    ("精心内", "心内(漢方）"): ["精神科", "心療内科"],#
    ("子どものこころ診療科", "子どものこころの診療科"): ['心療内科'],
    ("児精",  "老神"): ['精神科'],
    "心身医療科": ['心療内科'],
    ("心療", "老心療", "老心内", "女心内"): ['心療内科'],
   
    # ----血液・腫瘍内科----
    ("血液", "血"): ['血液・腫瘍内科'],

    # ----リハビリ----
    ("ﾃｰｼｮﾝ科", "−ション科"): ['リハビリテーション科'],
    "リバビリテーション科": ["リハビリテーション科"],

    # ----リウマチ----
    "リ内": ['リウマチ科'],

    # ----産婦人科----
    ("女性不妊症", "周内"): ['産婦人科'], # 周内 → 周産期内科 (Perinatal Internal Medicine)??
    
    # ----肝臓・胆嚢・膵臓----
    ("肝臓科", "肝臓", "肝"): ['肝・胆・膵内科', '肝・胆・膵外科'],
    ("胆のう", "胆の", "胆", "胆嚢"): ['肝・胆・膵内科', '肝・胆・膵外科'],
    ("膵臓", "膵"): ['肝・胆・膵内科', '肝・胆・膵外科'],
    ("肝臓・胆のう・膵臓科", "肝胆"): ['肝・胆・膵内科', '肝・胆・膵外科'],
    ("肝臓・消化"): ["肝・胆・膵内科", "肝・胆・膵外科", "消化器内科", "消化器外科"],

    # ----糖尿病・代謝内科----
    ("体謝"): ["糖尿病・代謝内科"], # typo of 代謝(taisha)?

    # ----小児科----
    ("小", "こ", "こ内", "小内", "新内", "内小", "循小", "小(新)", "(新)内", "（循）小", "新小", "(新)小", "新生内"): ["小児科"],
    "小外": ["小児外科"],

    # ----ペインクリニック系----
    "ペ": ['麻酔科', "整形外科"],

    # ----癌----
    ("が内", "化内", "化内", "薬内"): ['臨床腫瘍科'], # おそらく「がん内科」   Oncology/Internal Medicine for Cancer Treatment?

    # ----歯科----
    ("矯科", "矯", "小矯", "口外"): ["歯科・口腔顎顔面外科"], #おそらく「矯正歯科」 Orthodontics?
    "矯葉": ["歯科・口腔顎顔面外科"], # what 葉 means?
    "噛み合わせ不良による頭痛": ['歯科・口腔顎顔面外科'],
    "頭": ['歯科・口腔顎顔面外科'],

    # ----泌尿器科----
    ("生殖医療", "（生殖医療）", "生殖治療"): ["泌尿器科"],
    ("必尿器科", "泌"): ["泌尿器科"],
    ("ひ", "女ひ", "小ひ", "（女）泌", "泌(人工透析）", "女性泌"): ["泌尿器科"], # 「ひ」は泌尿器科？ Urology?

    # ----腎臓・内分泌----
    ("内分内", "内内", "内分", "内秘内", "分泌内", "分内", "分", "内秘", "脂内", "（脂)内", "脂", "高血圧内分内", "内泌内"): ['腎臓・内分泌内科'],
    ("小内内"): ["腎臓・内分泌内科", "小児科"], # おそらく「小児内分泌内科」 Pediatric Endocrinology?
    ("腎", "腎臓", "腎臓病", "腎臓科", "分泌"): ['腎臓・内分泌内科', '腎臓・内分泌外科'],
    ("乳腺内", "乳内", "内乳", "乳甲外"): ["腎臓・内分泌内科"],
    ("乳腺", "「乳腺", "乳", "乳腺科", "胸部・乳"): ['腎臓・内分泌内科', '腎臓・内分泌外科'],
    ("消化器乳"): ["腎臓・内分泌内科", "腎臓・内分泌外科",  "消化器内科", "消化器外科"],
    ("糖代内", "糖内分内"): ['腎臓・内分泌内科', "糖尿病・代謝内科"],
    ("甲状腺診療科", "甲状腺", "甲状腺科"): ['腎臓・内分泌内科', '腎臓・内分泌外科'],
    "血液浄化": ['腎臓・内分泌内科'],
    ("透", "透析", "透析科", "人工透",  "人工透析", "人工透析科", "血液透析",  "人透", "腎透", "血透",  "透内", "人口透析科", "通院人工透析診療所", "循環器科（透析）", "透科", "腎（人工透析）", "血液透析科", "腹膜透析）", "外（人透）", "泌(透）", "泌(人工透析)") : ['腎臓・内分泌内科'],
    ("ひ（透）", "ひ(透)", "ひ(透）", '腎ひ', '腎（透）', '泌（人工透析）', "泌（透析）", "泌（人透）", "泌（透）"): ['腎臓・内分泌内科', "泌尿器科"], # 「ひ」は泌尿器科？ただひらがなで表現する意味がわからない。「透」の字があることから、透析？ Dialysis (Nephrology-related department)?
    "高内": ["腎臓・内分泌内科"], # おそらく「高血圧・内分泌内科」 Hypertension and Endocrinology?
    "小腎": ['小児科', "腎臓・内分泌内科"], # おそらく 小児腎臓内科 or 小児腎臓外科 Pediatric Nephrology or Pediatric Kidney Surgery?
    "内高精": ["腎臓・内分泌内科", "泌尿器科"], # おそらく 内分泌・高血圧・生殖内科 or 生殖医療科 (精 = 精巣機能：男性不妊？)    Endocrinology, Hypertension, and Reproductive Medicine / Reproductive Medicine (related to male infertility?)
    ("乳内分外", "乳内外", "乳・内外", "乳外", "内分外", "甲外", "内泌", "透外"): [ "腎臓・内分泌外科"], # おそらく 乳腺内分泌内科 or 乳腺内分泌外科 Breast Endocrinology or Breast Endocrine Surgery?

    # ----病理診断科----
    ("理", "臨検", "検診．検査業務"): ['病理診断科'], #???
    "成分献血": ['病理診断科'],#

    # ----others====
    ("疼", "疼痛", "疼内"): ['麻酔科', '整形外科', 'リハビリテーション科'],
    ("薬物療法"): ["臨床腫瘍科", "精神科", "緩和ケア科"],
    ("脊椎", "脊髄外", "脊椎外", "脊椎髄外", "股関節外来", "側弯外来") : ['整形外科'],
    ("もの忘れ科"): ['精神科'],
    ("リ", "ﾘ", "り"): ['リハビリテーション科', 'リウマチ科'], # depends. check the context.
    ("思春", "思春期"): ["小児科", "産婦人科", "腎臓・内分泌内科", "精神科"],
    ("形外", "形"): ['形成外科・美容外科'],

    "小外形泌こう": ['小児外科', '形成外科・美容外科', '泌尿器科', '大腸・肛門内科', '大腸・肛門外科'],
    "形泌": ['形成外科・美容外科', '泌尿器科'],
    "乳外肛外": ['腎臓・内分泌外科', '大腸・肛門外科'],
    "静瘤外": ['心臓・血管外科', '循環器外科'],
    "胃．消．肛": ['消化器内科', '消化器外科', '大腸・肛門内科', "大腸・肛門外科"],

    "老年": ['老年科'],

    ("外．内", "内外", "外内"): ['一般外科', '一般内科'],
    "内．形．泌": ['一般内科', '形成外科・美容外科', '泌尿器科'],

    ("性内", "性"): ['感染症内科'],

    ("殖医外", "女内殖医外"): ['泌尿器科', '産婦人科'],

    ("内．泌", "泌内"): ['一般内科', '泌尿器科'],
    "内．心内": ['一般内科', '心療内科'],
}

In [ ]:
# load data
uniq_dpt = pd.read_csv('2020.csv').iloc[:,1]
uniq_dpt = uniq_dpt.to_list()
uniq_dpt

In [ ]:
classified_keywords = classify_keywords(
    keywords=uniq_dpt,               # List of unique medical department keywords to be classified
    classification_dict=dict.classification_dict,  # Dictionary defining classification rules for keywords
    multiple=True,                   # If True, allows matching with multiple classifications
    override_map=override_map_2020,   # Dictionary containing individual classification rules
    override_match_type="exact"       # Matching type for override_map classification ("exact" means exact match)
)

# Display the results in a DataFrame
max_length = max(len(values) for values in classified_keywords.values())
data = {
    cat: values + [""] * (max_length - len(values))
    for cat, values in classified_keywords.items()
}
result_df = pd.DataFrame(data)
display(result_df)

In [ ]:
result_df.to_csv('../classification_results/2020_classified.csv', encoding='utf-8', index=True)